#First, get the list of postal codes from Wikipedia using BeautifulSoup 


In [29]:
 
import urllib2
from bs4 import BeautifulSoup
base_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, 'lxml')
My_table = soup_index.find('table',{'class':'wikitable sortable'})

In [ ]:
# the three columns of the Wikipedia table are put into lists A,B,C
#then a datafram,  df, is constructed with 'PostalCode','Borough','Neighborhood'
# borough = Not assigned '  is removed    

In [234]:
A=[]
B=[]
C=[]

soup_index.find_all('tr')
for row in soup_index.find_all('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True)) 
        B.append(cells[1].find(text=True)) 
        C.append(cells[2].find(text=True)) 
#import pandas to convert list to data frame
import pandas as pd
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C

df2=df[df.Borough != 'Not assigned']

Create a dictionary named x.    Keys will be the unique postal codes from our Wikipedia table.
Second entry will be the full neighborhood name.  This starts out as 'ZZZZ'


In [281]:
x = dict.fromkeys(cod2,'ZZZZ')
   

Now go through all rows of df2.  First , replace 'Neighborhood' with the Borough name if Neighborhood is not assigned.
Then, for each Postal Code, if its dictionary entry is invalid ('ZZZZ'), replace it with the name of the neighborhood.   If the postal code already has a neighborhood, concatenate additional neighborhoods to dictiionary entry

In [282]:
st='Not assigned\n'
for index, row in df2.iterrows():
    if df2['Neighborhood'][index] == st:
        df2.set_value(index,'Neighborhood',df2['Borough'][index])
    if x[df2['PostalCode'][index]]=='ZZZZ':
        x[df2['PostalCode'][index]]=df2['Neighborhood'][index]
    else:
        x[df2['PostalCode'][index]]=x[df2['PostalCode'][index]]+','+df2['Neighborhood'][index]

In [287]:
#get unique postal code and Boroughs#
df_boroughs=df2[['PostalCode','Borough']].drop_duplicates(subset=['PostalCode','Borough'])
#now add the dictionary values to get the long Neighborhood names #
df_boroughs['Neighborhood']= df_boroughs['PostalCode'].map(x)
df_boroughs

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge,Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens,Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson,Garden District"


In [256]:
#get postal codes from csv  geocoder did not work#
postalcodes= pd.read_csv("https://cocl.us/Geospatial_data")
postalcodes

In [ ]:
Merge  boroughs to lat and long  by postal code to get final frme, df_final

In [288]:
df_final=df_boroughs.merge(postalcodes, left_on='PostalCode', right_on='Postal Code')

In [289]:
df_final

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
7,M3B,North York,Don Mills North,M3B,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",M4B,43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",M5B,43.657162,-79.378937
